# Ragas Usage Demo (End-to-End)

Place your Crucible Ragas export file in this same `notebooks/` folder, then set `RAGAS_EXPORT_FILE` below.

In [ ]:
# Optional install
# %pip install -U ragas datasets pandas requests


In [ ]:
import json
from pathlib import Path

import pandas as pd
import requests
from datasets import Dataset

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall


In [ ]:
NOTEBOOK_DIR = Path.cwd()
RAGAS_EXPORT_FILE = 'crucible_rag_ragas_example.json'  # change this
RAGAS_EXPORT_PATH = NOTEBOOK_DIR / RAGAS_EXPORT_FILE

if not RAGAS_EXPORT_PATH.exists():
    raise FileNotFoundError(f'Put export file in notebooks folder and update name: {RAGAS_EXPORT_PATH}')

raw = json.loads(RAGAS_EXPORT_PATH.read_text())
raw.keys()


In [ ]:
APP_API_URL = 'http://localhost:8000/chat'  # change to your app endpoint

def call_app(question: str):
    resp = requests.post(APP_API_URL, json={'question': question}, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    if isinstance(data, dict) and 'answer' in data:
        return data.get('answer', ''), data.get('contexts', []) or []

    return str(data), []


In [ ]:
questions = raw['question']
ground_truth = raw['ground_truth']

answers = []
contexts = []

for q in questions:
    a, c = call_app(q)
    answers.append(a)
    contexts.append(c if isinstance(c, list) else [str(c)])

df = pd.DataFrame({
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': ground_truth,
})

dataset = Dataset.from_pandas(df)
results = evaluate(dataset=dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall])
results


In [ ]:
summary = results.to_pandas()
out_path = NOTEBOOK_DIR / 'ragas_results.csv'
summary.to_csv(out_path, index=False)
out_path
